# ETL

## 1.0 Extraccion de datos

Se partio de un dataset con el precio historico del bitcoin para los valores diarios.

Para la extraccion y constante actualizacion se utilizara la libreria de ccxt, siendo esta la libreria ccxt que funciona como puente entre mi codigo y las apis para poder extraer los datos.

Esta librería no solo nos dejara extraer los datos del Bitcoin, si no ademas que nos permite interactuar con múltiples exchanges de forma unificada, facilitando la obtención de información de mercados y operaciones.

In [53]:
import ccxt, yfinance
import pandas as pd
import numpy as np
import pyarrow
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
# analisis tecnico
import ta# pip install TA-Lib
print(ccxt.exchanges) # print a list of all available exchange classes

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseadvanced', 'coinbaseexchange', 'coinbaseinternational', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okx', 'onetrading', 'oxfun

In [54]:
exchange = ccxt.binance()

## 1.1 Data Diaria

In [55]:
df_1d = pd.read_csv("Data/datasets/btc_price_historical_1d.csv",dtype={'Último':'float64'})

In [56]:
df_1d.rename(columns={' var.':'var'},inplace=True)
df_1d['var'] = df_1d['var'].astype(dtype=float)
df_1d.rename(columns={'Último':'close'},inplace=True)
df_1d.rename(columns={'Fecha':'date'},inplace=True)
df_1d['date'] = pd.to_datetime(df_1d['date'])
df_1d.rename(columns={'Vol.':'volume'},inplace=True)
df_1d.rename(columns={'Apertura':'open'},inplace=True)
df_1d.rename(columns={'Máximo':'high'},inplace=True)
df_1d.rename(columns={'Mínimo':'low'},inplace=True)
#df_1d.drop(columns=['Vol.'],inplace=True)
df_1d['return'] = ((df_1d['close'] - df_1d['open']) / df_1d['open'])*100
df_1d['diff'] = df_1d['close'] - df_1d['open']
df_1d['volatility'] = df_1d['high'] - df_1d['low']

C:\Users\Pablo\AppData\Local\Temp\ipykernel_9272\1646380069.py:5: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_1d['date'] = pd.to_datetime(df_1d['date'])


In [57]:
df_1d['date'] = pd.to_datetime(df_1d['date'])
df_1d.sort_values(by='date',inplace=True,ascending=True)

In [66]:
def transform_value(value):
    #print(type(value))
    #if value.endswith('K'):
    if type(value)== str:
        #print(type(value))
        if value[-1:]=='K':
            valor = value[:-1]
            valor = float(valor) * 1000
            return valor
            #return float(value.replace(',', '.').replace('K', '')) * 1000
        elif value[-1:]=='M':
            valor = value[:-1]
            valor = float(valor) * 10000
            return valor
            #return float(value.replace(',', '.').replace('M', '')) * 10000
        elif value[-1:]=='B':
            valor = value[:-1]
            valor = float(valor) * 1000000
            return valor
            #return float(value.replace(',', '.').replace('B', '')) * 1000000
        else:
            return value
    else:
        return value
# Aplicar la función de transformación al DataFrame
df_1d['volume'] = df_1d['volume'].apply(transform_value)

In [72]:
#Índice de Fuerza Relativa (RSI)
window_14 = 14
window_28 = 28
df_1d['rsi'] = ta.momentum.RSIIndicator(close=df_1d['close'],window=window_14).rsi()
# Medias Moviles
df_1d['ma_5'] = df_1d['close'].rolling(window=5).mean()
df_1d['ma_20'] = df_1d['close'].rolling(window=20).mean()
df_1d['ma_100'] = df_1d['close'].rolling(window=100).mean()
# Bandas de Bollinger solo se usara para graficar
df_1d['MiddleBand'] = df_1d['close'].rolling(window=20).mean()
df_1d['UpperBand'] = df_1d['MiddleBand'] + 2*df_1d['close'].rolling(window=20).std()
df_1d['LowerBand'] = df_1d['MiddleBand'] - 2*df_1d['close'].rolling(window=20).std()
#Oscilador Estocástico
df_1d['%K'] = 100 * ((df_1d['close'] - df_1d['low'].rolling(window=14).min()) / (df_1d['high'].rolling(window=14).max() - df_1d['low'].rolling(window=14).min()))
df_1d['%D'] = df_1d['%K'].rolling(window=3).mean()
#Índice de Canal de Materias Primas (CCI)
df_1d['TP'] = (df_1d['high'] + df_1d['low'] + df_1d['close']) / 3
df_1d['CCI'] = (df_1d['TP'] - df_1d['TP'].rolling(window=20).mean()) / (0.015 * df_1d['TP'].rolling(window=20).std())

## 1.2 Data 4 horas

In [58]:
bars = exchange.fetch_ohlcv('BTC/USDT', timeframe='4h', limit=500)
df_4h = pd.DataFrame(bars, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
ultimo_timestamp = df_4h['time'][0]
start_timestamp = ultimo_timestamp
# Convertimos 2 meses a milisegundos
two_months_ms = 60 * 24 * 60 * 60 * 1000  # 60 días * 24 horas * 60 minutos * 60 segundos * 1000 milisegundos
#four_hours_ms = 4 * 60 * 60 * 1000# Convertimos 4 horas a milisegundos
all_data = []
# Determina cuántos periodos de 2 meses necesitas para retroceder 5 años
periods = 5 * 365 // 60  # 5 años en períodos de 2 meses
for i in range(1, periods + 1):  # Ajusta el rango según cuántos periodos de 2 meses quieras retroceder
    # Ajustar el número de velas para 2 meses (60 días)
    new_data = exchange.fetch_ohlcv('BTC/USDT', timeframe='4h', since=start_timestamp, limit=60 * 24 // 4)  # 60 días * 24 horas / 4 horas por vela
    #time.sleep(2)
    all_data.extend(new_data)
    # Desplazar el timestamp por el siguiente periodo de 2 meses multiplicado por i
    start_timestamp -= two_months_ms
# Crear DataFrame con todos los datos obtenidos
df2 = pd.DataFrame(all_data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df2 = df2.sort_values(by='time')
df_4h = pd.merge(df2, df_4h, how='outer')
#df_4h.drop_duplicates(subset=['time'],inplace=True)

last_timestamp = df_4h['time'].iloc[-1]
last_timestamp = float(last_timestamp)

known_date = datetime.strptime("2024/07/23", "%Y/%m/%d")
known_timestamp = df_4h['time'].tail(1).values
known_timestamp = float(known_timestamp[0])
today = datetime.now()
df_4h['date'] = df_4h['time'].apply(lambda x: today - timedelta(milliseconds=(last_timestamp - x)))
df_4h['date'] = df_4h['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

#ordenar los datos
df_4h.sort_values(by='time',axis=0,inplace=True)
df_4h.reset_index(inplace=True)
df_4h.drop(columns=['index'],inplace=True)

df_4h['return'] = ((df_4h['close'] - df_4h['open']) / df_4h['open'])*100
df_4h['diff'] = df_4h['close'] - df_4h['open']
df_4h['volatility'] = df_4h['high'] - df_4h['low']

In [59]:
#Índice de Fuerza Relativa (RSI)
window_14 = 14
window_28 = 28
df_4h['rsi'] = ta.momentum.RSIIndicator(close=df_4h['close'],window=window_14).rsi()
# Medias Moviles
df_4h['ma_5'] = df_4h['close'].rolling(window=10).mean()
df_4h['ma_20'] = df_4h['close'].rolling(window=10).mean()
df_4h['ma_100'] = df_4h['close'].rolling(window=50).mean()
# Bandas de Bollinger solo se usara para graficar
df_4h['MiddleBand'] = df_4h['close'].rolling(window=20).mean()
df_4h['UpperBand'] = df_4h['MiddleBand'] + 2*df_4h['close'].rolling(window=20).std()
df_4h['LowerBand'] = df_4h['MiddleBand'] - 2*df_4h['close'].rolling(window=20).std()
#Oscilador Estocástico
df_4h['%K'] = 100 * ((df_4h['close'] - df_4h['low'].rolling(window=14).min()) / (df_4h['high'].rolling(window=14).max() - df_4h['low'].rolling(window=14).min()))
df_4h['%D'] = df_4h['%K'].rolling(window=3).mean()
#Índice de Canal de Materias Primas (CCI)
df_4h['TP'] = (df_4h['high'] + df_4h['low'] + df_4h['close']) / 3
df_4h['CCI'] = (df_4h['TP'] - df_4h['TP'].rolling(window=20).mean()) / (0.015 * df_4h['TP'].rolling(window=20).std())

#### Las columnas que se utilizaran para el panel de informacion seran:

- Rsi: Indice de fuerza relativa
- K y D: Componentes del Oscilador estocastico
- CCI: Indice del canal de materias primas

## 1.3 Data 5 minutos

In [60]:
bars = exchange.fetch_ohlcv('BTC/USDT',timeframe='5m',limit=288)
df_5m = pd.DataFrame(bars, columns=['time','open','high','low','close','volume'])
#ultimo_timestamp = df['time'][0]
primer_timestamp = df_5m.iloc[-1,0]
start_timestamp = primer_timestamp
# Convertimos un año a milisegundos
#one_year_ms = 365 * 86400000
#one_hour = 60*60*1000
one_day = 24*60*60*1000
#four_hours_ms = 4 * 60 * 60 * 1000# Convertimos 4 horas a milisegundos
all_data = []
for i in range(1,60): # stop = cantidad de horas
    new_data = exchange.fetch_ohlcv('BTC/USDT', timeframe='5m', since=start_timestamp, limit=288)
    time.sleep(1)
    all_data.extend(new_data)
    # Desplazar el timestamp por el siguiente año multiplicado por i
    start_timestamp -= one_day
# Crear DataFrame con todos los datos obtenidos
df_5m = pd.DataFrame(all_data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df_5m = df_5m.sort_values(by='time').reset_index(drop=True)
#df_5m.drop_duplicates(inplace=True)

df_5m['return'] = ((df_5m['close'] - df_5m['open']) / df_5m['open'])*100
df_5m['diff'] = df_5m['close'] - df_5m['open']
df_5m['volatility'] = df_5m['high'] - df_5m['low']

known_date = datetime.strptime("2024/07/30", "%Y/%m/%d")
known_timestamp = 1703808000000
today = datetime.now()
print(today)
restar_un_dia = timedelta(days=1)
# Obtener la fecha de ayer
ayer = today - restar_un_dia
last_timestamp = df_5m['time'].iloc[-1]
last_timestamp = float(last_timestamp)
df_5m['date'] = df_5m['time'].apply(lambda x: today - timedelta(milliseconds=(last_timestamp - x)))
df_5m['date'] = df_5m['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

2024-09-10 17:01:29.889562


In [61]:
#Índice de Fuerza Relativa (RSI)
window_14 = 14
window_28 = 28
df_5m['rsi'] = ta.momentum.RSIIndicator(close=df_5m['close'],window=window_14).rsi()
# Medias Moviles
df_5m['ma_5'] = df_5m['close'].rolling(window=10).mean()
df_5m['ma_20'] = df_5m['close'].rolling(window=10).mean()
df_5m['ma_100'] = df_5m['close'].rolling(window=50).mean()
# Bandas de Bollinger solo se usara para graficar
df_5m['MiddleBand'] = df_5m['close'].rolling(window=20).mean()
df_5m['UpperBand'] = df_5m['MiddleBand'] + 2*df_5m['close'].rolling(window=20).std()
df_5m['LowerBand'] = df_5m['MiddleBand'] - 2*df_5m['close'].rolling(window=20).std()
#Oscilador Estocástico
df_5m['%K'] = 100 * ((df_5m['close'] - df_5m['low'].rolling(window=14).min()) / (df_5m['high'].rolling(window=14).max() - df_5m['low'].rolling(window=14).min()))
df_5m['%D'] = df_5m['%K'].rolling(window=3).mean()
#Índice de Canal de Materias Primas (CCI)
df_5m['TP'] = (df_5m['high'] + df_5m['low'] + df_5m['close']) / 3
df_5m['CCI'] = (df_5m['TP'] - df_5m['TP'].rolling(window=20).mean()) / (0.015 * df_5m['TP'].rolling(window=20).std())

## 1.4 Data 1 hora

In [62]:
bars = exchange.fetch_ohlcv('BTC/USDT', timeframe='1h', limit=500)
df_1h = pd.DataFrame(bars, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
ultimo_timestamp = df_1h['time'][0]
start_timestamp = ultimo_timestamp
two_weeks_ms = 2*7*24 * 60 *60 * 1000
all_data = []
periods = 2 * 365 // 14  # 5 años en períodos de 2 meses
for i in range(1, periods + 1):  # Ajusta el rango según cuántos periodos de 2 meses quieras retroceder
    # Ajustar el número de velas para 2 meses (60 días)
    new_data = exchange.fetch_ohlcv('BTC/USDT', timeframe='1h', since=start_timestamp, limit=360)
    #time.sleep(2)
    all_data.extend(new_data)
    # Desplazar el timestamp por el siguiente periodo de 2 meses multiplicado por i
    start_timestamp -= two_weeks_ms
df2 = pd.DataFrame(all_data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df2 = df2.sort_values(by='time')
df_1h = pd.merge(df2, df_1h, how='outer')
last_timestamp = df_1h['time'].iloc[-1]
last_timestamp = float(last_timestamp)

known_date = datetime.strptime("2024/07/23", "%Y/%m/%d")
known_timestamp = df_1h['time'].tail(1).values
known_timestamp = float(known_timestamp[0])
today = datetime.now()
df_1h['date'] = df_1h['time'].apply(lambda x: today - timedelta(milliseconds=(last_timestamp - x)))
df_1h['date'] = df_1h['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

#ordenar los datos
df_1h.sort_values(by='time',axis=0,inplace=True)
df_1h.reset_index(inplace=True)
df_1h.drop(columns=['index'],inplace=True)

df_1h['return'] = ((df_1h['close'] - df_1h['open']) / df_1h['open'])*100
df_1h['diff'] = df_1h['close'] - df_1h['open']
df_1h['volatility'] = df_1h['high'] - df_1h['low']

In [63]:
#Índice de Fuerza Relativa (RSI)
window_14 = 14
window_28 = 28
df_1h['rsi'] = ta.momentum.RSIIndicator(close=df_1h['close'],window=window_14).rsi()
# Medias Moviles
df_1h['ma_5'] = df_1h['close'].rolling(window=10).mean()
df_1h['ma_20'] = df_1h['close'].rolling(window=10).mean()
df_1h['ma_100'] = df_1h['close'].rolling(window=50).mean()
# Bandas de Bollinger solo se usara para graficar
df_1h['MiddleBand'] = df_1h['close'].rolling(window=20).mean()
df_1h['UpperBand'] = df_1h['MiddleBand'] + 2*df_1h['close'].rolling(window=20).std()
df_1h['LowerBand'] = df_1h['MiddleBand'] - 2*df_1h['close'].rolling(window=20).std()
#Oscilador Estocástico
df_1h['%K'] = 100 * ((df_1h['close'] - df_1h['low'].rolling(window=14).min()) / (df_1h['high'].rolling(window=14).max() - df_1h['low'].rolling(window=14).min()))
df_1h['%D'] = df_1h['%K'].rolling(window=3).mean()
#Índice de Canal de Materias Primas (CCI)
df_1h['TP'] = (df_1h['high'] + df_1h['low'] + df_1h['close']) / 3
df_1h['CCI'] = (df_1h['TP'] - df_1h['TP'].rolling(window=20).mean()) / (0.015 * df_1h['TP'].rolling(window=20).std())

In [73]:
df_1d.rename(columns={'%K':'K','%D':'D'},inplace=True)
df_4h.rename(columns={'%K':'K','%D':'D'},inplace=True)
df_1h.rename(columns={'%K':'K','%D':'D'},inplace=True)
df_5m.rename(columns={'%K':'K','%D':'D'},inplace=True)

In [29]:
df_5m = df_5m[:-1000]
df_1h = df_1h[:-1000]
df_4h = df_4h[:-1000]
df_1d = df_1d[:-1000]

In [76]:
print(len(df_1d.columns))
print(len(df_4h.columns))
print(len(df_1h.columns))
print(len(df_5m.columns))

21
21
21
21


In [77]:
df_5m.to_parquet('Data/datasets/btc_5m.parquet',engine='pyarrow')
df_1h.to_parquet('Data/datasets/btc_1h.parquet',engine='pyarrow')
df_4h.to_parquet('Data/datasets/btc_4h.parquet',engine='pyarrow')
df_1d.to_parquet('Data/datasets/btc_1d.parquet',engine='pyarrow')